#### Movie Recommendation using Deeplearning

1. https://github.com/Tejas-TA/Neural-Network-Movie-Recommendation/blob/main/Neural_Recommendation_System.ipynb
2. https://github.com/rajendra7406/movie_recommender/blob/master/movie%20recommenders.ipynb

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Model
import tensorflow as tf
from keras.layers import Dense, Input,Embedding, Dropout, Flatten, concatenate
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


Loading Data

In [9]:
use_large = False
if use_large:
    movies = pd.read_csv("../../../../DataSets/ml-latest/movies.csv")
    ratings = pd.read_csv("../../../../DataSets/ml-latest/ratings.csv")
    tags = pd.read_csv("../../../../DataSets/ml-latest/tags.csv")
    links = pd.read_csv("../../../../DataSets/ml-latest/links.csv")
else:
    movies = pd.read_csv("../../../../DataSets/ml-latest-small/movies.csv")
    ratings = pd.read_csv("../../../../DataSets/ml-latest-small/ratings.csv")
    tags = pd.read_csv("../../../../DataSets/ml-latest-small/tags.csv")
    links = pd.read_csv("../../../../DataSets/ml-latest-small/links.csv")


In [10]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [11]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

#count of users, movies
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
'users =', n_users, 'movies =',  n_movies


('users =', 610, 'movies =', 9724)

In [12]:
userid2idx = { o:i for i,o in enumerate(users) }
movieid2idx = {o:i for i,o in enumerate(movies) }

In [13]:
ratings.userId = ratings.userId.apply(lambda x:userid2idx[x])
ratings.movieId = ratings.movieId.apply(lambda x:movieid2idx[x])

In [14]:
ratings.head()


,userId,movieId,rating,timestamp
0,0,0,4.0,964982703
1,0,1,4.0,964981247
2,0,2,4.0,964982224
3,0,3,5.0,964983815
4,0,4,5.0,964982931


In [15]:
n_factors = 50
np.random.seed = 42


In [16]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]


In [17]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
movie_in = Input(shape=(1,), dtype='int64', name='movies_in')


In [18]:
u = Embedding(n_users, n_factors, input_length=1)(user_in)
m = Embedding(n_movies, n_factors, input_length=1)(movie_in)


In [19]:
x = concatenate([u, m])
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70,activation='relu')(x)
x = Dropout(0.7)(x)
x = Dense(1)(x)
nn = Model([user_in,movie_in],x)

In [20]:
nn.compile(optimizer=Adam(0.01), loss='mse', metrics=['accuracy'])

In [21]:
nn.fit([trn.userId, trn.movieId], trn.rating,
       epochs=2,
       batch_size=64,
       validation_data=([val.userId, val.movieId], val.rating)
       )


Epoch 1/2
1256/1256 [==============================] - 10s 7ms/step - loss: 1.2772 - accuracy: 0.0277 - val_loss: 0.8125 - val_accuracy: 0.0280
Epoch 2/2
1256/1256 [==============================] - 9s 7ms/step - loss: 0.8530 - accuracy: 0.0278 - val_loss: 0.8102 - val_accuracy: 0.0280


In [22]:
nn.compile(optimizer=Adam(0.001), loss='mse', metrics=['accuracy'])


In [23]:
nn.fit([trn.userId, trn.movieId], trn.rating,
       epochs=2,
       batch_size=64,
       validation_data=([val.userId, val.movieId], val.rating)
       )


Epoch 1/2
1256/1256 [==============================] - 10s 7ms/step - loss: 0.7355 - accuracy: 0.0278 - val_loss: 0.7714 - val_accuracy: 0.0280
Epoch 2/2
1256/1256 [==============================] - 8s 6ms/step - loss: 0.7099 - accuracy: 0.0277 - val_loss: 0.7600 - val_accuracy: 0.0280


Changing learning rate gave a bump in accuracy.

In [24]:
x1 = concatenate([u,m])
x1 = Flatten()(x1)
x1 = Dense(70,activation='relu')(x1)
x1 = Dropout(0.5)(x1)
x1 = Dense(1)(x1)
nn = Model([user_in,movie_in],x1)

In [25]:
nn.compile(optimizer=Adam(0.001),loss='mse')

In [26]:
X_train = [trn.userId, trn.movieId]
y_train = trn.rating

In [27]:
nn.fit(
    X_train, y_train,
    epochs=2,
    batch_size=64,
    validation_data=([val.userId, val.movieId], val.rating)
)


Epoch 1/2
1256/1256 [==============================] - 10s 7ms/step - loss: 1.6209 - val_loss: 0.8101
Epoch 2/2
1256/1256 [==============================] - 11s 9ms/step - loss: 0.9343 - val_loss: 0.7866


In [28]:
nn.predict([np.array([5]), np.array([10])])


1/1 [==============================] - 0s 166ms/step


array([[3.6342998]], dtype=float32)